In [1]:
import time
t=time.time()
import random

CARSPEED=2
WALKERSPEED=5
WIDTH=5
CHANGETIME=3
TACT=10

sat =[[1,10,],[1,4],[7,10],[7,4],[0,9],[0,3],[0,6],[9,3],[3,7],[6,9]
     ,[0,10],[0,7],[9,7],[9,4],[6,4],[6,1],[3,1],[3,10],[12,1],[13,10]
     ,[14,11],[14,7],[15,0],[15,4]]

for i in range(5,10):
    sat.append([12,i])
for i in range(2,7):
    sat.append([13,i])
for i in range(0,4):
    sat.append([14,i])
for i in range(8,12):
    sat.append([15,i])
    
times=[]
zone=[]
blockzone=[]
prevmask=[]

contflow=[]
for i in range(16):
    contflow.append(TACT//(CARSPEED*(i<12)+(i>=12)*WALKERSPEED))
for i in range(12,16):
    contflow[i]*=WIDTH

changeflow=[]
for i in range(16):
    changeflow.append((TACT-CHANGETIME)//(CARSPEED*(i<12)+(i>=12)*WALKERSPEED))
for i in range(12,16):
    changeflow[i]*=WIDTH

weights=[]
for i in range(16):
    weights.append(1)
for i in range(12,16):
    weights[i]/=3

timeweights=[]
for i in range(16):
    timeweights.append(1)
for i in range(12):
    timeweights[i]/=2    

print(weights)
print(timeweights)
print(contflow)
print(changeflow)    
    

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 1]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5]


In [2]:
def possiblemask(mask):
    for i in sat:
        if not ((mask[i[0]]==0) or (mask[i[1]]==0)):
            return 0
    return 1

In [3]:
def genmask():
    a=0
    while(not a): 
        mask=[]
        for i in range(16):
            mask.append(random.randint(0,1))
        a=possiblemask(mask)
    return mask
prevmask=genmask()

In [4]:
def randdata(ch):
    global blockzone
    global zone
    global times
    global prevmask
    blockzone=[]
    for i in range(4):
        blockzone.append((random.randint(0,3)==2)*random.randint(0,4))
    zone=[]
    for i in range(16):
        zone.append(random.randint(0,4)+(i>12)*(random.randint(0,5)))
    if ch:
        prevmask=genmask()
        times=[]
        for i in range(16):
            if zone[i] and not prevmask[i]:
                times.append(random.randint(1,60))
            else:
                times.append(1)

In [5]:
def tomask(a):
    i=0
    mask=[]
    for i in range(16):
        mask.append(a%2)
        a//=2
    return mask

In [6]:
def greed(mask,zone):
    ans=0
    for i in range(16):
        if(prevmask[i]==mask[i]):
            ans+=max(zone[i],contflow[i])*mask[i]*weights[i]*(((times[i]*timeweights[i])/30)**2)
        else:
            ans+=max(zone[i],changeflow[i])*mask[i]*weights[i]*(((times[i]*timeweights[i])/30)**2)
    return ans

In [7]:
def noblock(mask,blockzone):
    blockmask=[]
    for i in range(12):
        blockmask.append(0)
    for j in range(0,4):
        if(blockzone[j]):
            for i in range(j*3,3+j*3):
                blockmask[i]=1
            for i in range(j*3+3,j*3+11,4):
                blockmask[i%12]=1
    for i in range(12):
        if(mask[i] and blockmask[i]):
            return 0
    return 1

In [8]:
pt=time.time()

def greedchoose(masksize, zone):
    maxv=0
    maxmask=[]
    for i in range(1 << masksize):
        mask = tomask(i+1)
        a = greed(mask, zone) * possiblemask(mask)*noblock(mask,blockzone)
        if(maxv <= a):
            maxv = a
            maxmask = mask
    for i in range(16):
        if (maxmask[i]):
            times[i]=1
        elif (zone[i]):
            times[i]+=TACT
    return [maxmask, maxv]

In [11]:
randdata(1)
for i in range(10):
    pt=time.time()
    print(i,":times",times)
    print(i,":timeweights",timeweights)
    print()
    print(i,":zone",zone)
    print(i,":weights",weights)
    print()
    print(i,":blockzone",blockzone)
    print(i,":prevmask",prevmask)
    a=greedchoose(16, zone)
    print(i,":conclusion",a)
    print(i,":times",times)
    print(i,":global time",time.time()-t)
    print(i,":iteration time",time.time()-pt)
    print()
    randdata(0)
    prevmask=a[0]

0 :times [47, 7, 1, 12, 38, 1, 1, 54, 1, 18, 1, 36, 11, 1, 35, 9]
0 :timeweights [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 1]

0 :zone [3, 4, 3, 4, 1, 0, 4, 4, 0, 1, 0, 3, 2, 0, 1, 6]
0 :weights [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

0 :blockzone [0, 0, 0, 0]
0 :prevmask [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 :conclusion [[0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0], 4.378055555555556]
0 :times [57, 1, 1, 22, 48, 1, 11, 1, 1, 28, 1, 1, 21, 1, 45, 19]
0 :global time 520.9309031963348
0 :iteration time 1.9628734588623047

1 :times [57, 1, 1, 22, 48, 1, 11, 1, 1, 28, 1, 1, 21, 1, 45, 19]
1 :timeweights [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1, 1, 1, 1]

1 :zone [0, 4, 1, 3, 0, 2, 3, 0, 2, 3, 2, 4, 4, 5, 9, 3]
1 :weights [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

1 :blockzo

In [10]:
mask=[]
for i in range(16):
    mask.append(0)
noblock(mask,blockzone)

1